## Importing Libraries

In [12]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from sklearn.preprocessing import MinMaxScaler,StandardScaler

## Data Prepare

In [14]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
df = train_data[['Survived', 'Sex','Age','Fare','Pclass']]
df['Sex'].replace(['male','female'],[1,0], inplace=True)

C:\Users\E1385584\AppData\Local\Temp\ipykernel_8128\2976133309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'].replace(['male','female'],[1,0], inplace=True)


In [19]:
mean_age = np.mean(df['Age'])

In [21]:
df['Age'].fillna(mean_age, inplace=True)

C:\Users\E1385584\AppData\Local\Temp\ipykernel_8128\3636672125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'].fillna(mean_age, inplace=True)


In [33]:
X=df[['Sex','Age','Fare','Pclass']].values
y=df[['Survived']].values

In [34]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X)

In [36]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y)

In [37]:
y_tensor = y_tensor.squeeze(dim=1)
y_tensor = y_tensor.type(torch.LongTensor)

## Model And Train

In [38]:
class Net(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=128,out_features=out_dim)
        )
    def forward(self,X):
        X = self.layer(X)
        return X

In [39]:
model = Net(in_dim=4,out_dim=2)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),0.025)

In [41]:
for i in range(10000):
    acc = 0
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred,y_tensor)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    if (i + 1) % 1000 == 0:
        print(f'Epoch : {i + 1}\nLoss : {loss}')

Epoch : 1000
Loss : 0.44738611578941345
Epoch : 2000
Loss : 0.4353536069393158
Epoch : 3000
Loss : 0.4286348521709442
Epoch : 4000
Loss : 0.4233807325363159
Epoch : 5000
Loss : 0.42479774355888367
Epoch : 6000
Loss : 0.4238097667694092
Epoch : 7000
Loss : 0.4211091697216034
Epoch : 8000
Loss : 0.42526134848594666
Epoch : 9000
Loss : 0.4231555163860321
Epoch : 10000
Loss : 0.42268016934394836


## Testing

In [42]:
test_df = pd.read_csv('test.csv')
df = test_df[[ 'Sex','Age','Fare','Pclass']]
df['Sex'].replace(['male','female'],[1,0], inplace=True)
mean_age = np.mean(df['Age'])
df['Age'].fillna(mean_age, inplace=True)
mean_fare = np.mean(df['Fare'])
df['Fare'].fillna(mean_fare, inplace=True)

C:\Users\E1385584\AppData\Local\Temp\ipykernel_8128\3027685125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'].replace(['male','female'],[1,0], inplace=True)
C:\Users\E1385584\AppData\Local\Temp\ipykernel_8128\3027685125.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'].fillna(mean_age, inplace=True)
C:\Users\E1385584\AppData\Local\Temp\ipykernel_8128\3027685125.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [43]:
X = df[['Sex','Age','Fare','Pclass']].values
scaler = MinMaxScaler()
X_ = scaler.fit_transform(X)
X_t = torch.tensor(X_, dtype=torch.float32)

In [44]:
model.eval()
y_pred = model(X_t).argmax(1)

In [45]:
ids = test_df['PassengerId'].values

In [46]:
dictt = {'PassengerId':ids, 'Survived':y_pred}
df = pd.DataFrame(dictt)
df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
